In [32]:
!pip install langchain_nvidia_ai_endpoints -q

In [33]:
!pip install langchain_community -q

In [34]:
!pip install langchain-community langchain-text-splitters faiss-cpu -q

In [7]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA

In [8]:
import os

In [9]:
from langchain_community.document_loaders import WebBaseLoader

In [10]:
#loader = WebBaseLoader("https://medium.com/@artgor/paper-review-personalized-audiobook-recommendations-at-spotify-through-graph-neural-networks-bd24cf0dd198")
loader = WebBaseLoader("https://medium.com/@artgor/paper-review-personalized-audiobook-recommendations-at-spotify-through-graph-neural-networks-bd24cf0dd198")

In [11]:
docs=loader.load()

In [12]:
from google.colab import userdata
os.environ['NVIDIA_API_KEY']=userdata.get('google_gemma')

In [13]:
embeddings= NVIDIAEmbeddings()

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [15]:
text_splitter= RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [16]:
document=text_splitter.split_documents(docs)

In [17]:
document[0]

Document(page_content='Paper Review: Personalized Audiobook Recommendations at Spotify Through Graph Neural Networks | by Andrew Lukyanenko | Mar, 2024 | MediumOpen in appSign upSign inWriteSign upSign inPaper ReviewPaper Review: Personalized Audiobook Recommendations at Spotify Through Graph Neural NetworksRecommendation system for Audiobooks using GNN!Andrew Lukyanenko·Follow6 min read·Mar 19, 2024--ListenSharePaper linkSpotify has recently expanded its offerings to include audiobooks, presenting challenges in', metadata={'source': 'https://medium.com/@artgor/paper-review-personalized-audiobook-recommendations-at-spotify-through-graph-neural-networks-bd24cf0dd198', 'title': 'Paper Review: Personalized Audiobook Recommendations at Spotify Through Graph Neural Networks | by Andrew Lukyanenko | Mar, 2024 | Medium', 'description': 'Spotify has recently expanded its offerings to include audiobooks, presenting challenges in personalized recommendation due to the inability to skim audiobook

In [18]:
vector=FAISS.from_documents(document,embeddings)

In [19]:
reteiver=vector.as_retriever()

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [21]:
model = ChatNVIDIA(model="mistral_7b")

In [22]:
hyde_template = """Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:

{question}"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_query_transformer = hyde_prompt | model | StrOutputParser()

In [23]:
hyde_prompt

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:\n\n{question}'))])

In [24]:
from langchain_core.runnables import chain

In [25]:
@chain
def hyde_ret(question):
  hypo_document= hyde_query_transformer.invoke({"question":question})
  return retriever.invoke(hypo_document)

In [26]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
answer_chain = prompt | model | StrOutputParser()

In [27]:
@chain
def final_chain(question):
    documents = hyde_query_transformer.invoke(question)
    for s in answer_chain.stream({"question": question, "context": documents}):
        yield s

In [29]:
for s in final_chain.stream("what are the course of Panimalar Engineering College"):
    print(s, end="")

Panimalar Engineering College offers undergraduate and postgraduate programs in various engineering disciplines. The following are some of the courses provided by the college:

1. Computer Science and Engineering
2. Electronics and Communication Engineering
3. Mechanical Engineering
4. Civil Engineering
5. Information Technology

These courses are designed to provide students with a strong foundation in their chosen field and prepare them for successful careers in engineering and technology. The college also offers extra-curricular activities, industrial visits, and research opportunities to enhance the overall learning experience of students.

In [31]:
for s in final_chain.stream("who is the Information Technology's HOD?"):
    print(s, end="")

The Information Technology (IT) Head of Department (HOD) is an experienced and qualified IT professional responsible for managing and leading the IT team, implementing IT strategies, and ensuring the effective and efficient use of technology to support the organization's goals. They have a strong background in IT, a deep understanding of the latest technologies and trends in the field, excellent communication and leadership skills, and are dedicated to using technology to drive innovation, improve processes, and provide value to the organization.